# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのAIアシスタントを構築するためのものです。主に、言葉当てゲーム「20の質問」をプレイできるモデルを開発し、質問者と回答者の役割を果たすことで、できるだけ少ない質問でターゲットを当てることを目指しています。

### 問題
このNotebookでは、20の質問ゲームにおいて、回答者が考えたキーワードに対して質問者が質問し、その質問に基づいて推測を行うというタスクに取り組んでいます。具体的には、与えられた質問に対する回答をモデルが生成し、適切にキーワードを推測する役割を果たす必要があります。

### 手法
1. **環境設定**: 必要なライブラリ（`immutabledict`、`sentencepiece`、`kaggle_environments`）をインストールし、GemmaのPyTorch実装をクローンして、作業ディレクトリに配置します。
  
2. **モデルの構成**: 
    - `GemmaFormatter`クラスを使用して、ユーザーとモデルのターンをフォーマットします。
    - 構成情報に基づいてGemmaモデルをロードし、異なるバリエーション（例: "2b"、"2b-it"、"7b-it-quant"）でモデルを初期化します。

3. **プロンプトの準備**: 
    - ゲームのシステムプロンプトや過去の例（few-shot examples）を用いて、質問と回答のターンを整列させるためのフォーマッターを使用します。

4. **応答生成**: 
    - モデルを用いて生成されたプロンプトに基づいて、質問や推測の応答を生成します。

5. **応答解析**: 
    - 抽出したキーワードや生成した質問を解析し、ゲームの流れに従って答えを導き出します。

### 使用ライブラリ
- `kaggle_environments`: Kaggleの競技環境を構築するために利用。
- `torch`: PyTorchを利用して深層学習モデルを実装。
- `gemma`: Googleが開発したGemmaモデルの実装を使用。

このNotebookは、言語モデルが「20の質問」ゲームを通じて有用な質問を生成し、論理的に推論してキーワードを当てる能力を評価し、改善するための基盤を築いています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する初心者がつまずく可能性のある専門用語の解説です。

1. **GemmaForCausalLM**:
   - Causal Language Model（因果言語モデル）の一種で、入力に基づいて次の単語を生成するために訓練されたモデルです。この種のモデルは、文脈に基づいて単語を予測するため、対話や物語生成などに使用されます。

2. **Tokenizer**:
   - テキストデータをモデルが理解できる形式（通常は数値のリスト）に変換するためのツールです。自然言語処理では、単語やトークンを分割して表現するために使用されます。

3. **Few-shot Examples**:
   - モデルに少数の例を与えることで、その後のタスクをより良く理解させる手法です。特に、言語モデルに対話の流れや形式を教える際に有用です。

4. **Temperature**:
   - モデルが生成する出力の多様性をコントロールするパラメータです。温度が高いと出力は多様になり、低いと出力が確定的になりやすいです。このパラメータを調整することで、生成される文章の創造性を操作できます。

5. **Top-p Filtering (Nucleus Sampling)**:
   - 確率が高い候補トークンを選択し、それらの合計確率が事前に指定された値（p）を超えるまでトークンを選ぶ方法です。これにより、出力がより自然で関連性のあるものになることが期待されます。

6. **Top-k Filtering**:
   - 出力候補としてk個の最も高い確率を持つトークンの中から新しいトークンを生成する方法です。これにより、無関係なトークンが選択される確率が減り、より洗練された結果が得られます。

7. **Contextlib**:
   - Pythonの標準ライブラリで、リソースの管理（例：ファイルの開閉）を簡単に行うためのツールです。このノートブックでは、コンテキストマネージャを使用して、デフォルトのデータ型を設定しています。

8. **Quantization**:
   - モデルのサイズを小さくし、計算速度を向上させるために、モデルの重みを圧縮して低精度の数値形式に変換する技術です。これにより、リソースの少ない環境でもモデルを実行できるようになります。

9. **Interleave**:
   - 二つのリストやシーケンスを交互に統合するプロセスです。このノートブックでは、質問と回答を一つのシーケンスにするために使われています。

10. **Observation (`obs`)**:
    - モデルが現在のゲームの状態を知るために使用する情報の集合です。この情報には、ターンの種類や以前の質問と回答が含まれます。

これらの用語の理解は、このノートブックが持つ特有のドメイン知識や機能を理解するために重要となります。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!pip install -q 'kaggle_environments>=1.14.8'
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/
```

</div>
<div class="column-right">

# 日本語訳

```python
# 環境を設定する
!pip install -q -U immutabledict sentencepiece 
!pip install -q 'kaggle_environments>=1.14.8'
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/
```

</div>
</details>

In [ ]:
# 環境を設定する
!pip install -q -U immutabledict sentencepiece 
!pip install -q 'kaggle_environments>=1.14.8'
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
import contextlib
import os
import torch
import re
import kaggle_environments
import itertools
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from typing import Iterable
```

</div>
<div class="column-right">

# 日本語訳

```python
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
import contextlib
import os
import torch
import re
import kaggle_environments
import itertools
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from typing import Iterable
```

</div>
</details>

In [ ]:
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
import contextlib
import os
import torch
import re
import kaggle_environments
import itertools
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from typing import Iterable

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self
```

</div>
<div class="column-right">

# 日本語訳

```python
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt  # システムプロンプトを保存
        self._few_shot_examples = few_shot_examples  # 例を保存
        # ユーザーとモデルのターンをフォーマットするためのテンプレート
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()  # 初期状態にリセット

    def __repr__(self):
        return self._state  # 現在の状態を返す

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)  # ユーザーのプロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)  # モデルのプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"  # ユーザーのターンを開始
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"  # モデルのターンを開始
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"  # ターンを終了
        return self

    def reset(self):
        self._state = ""  # 状態をクリア
        if self._system_prompt is not None:
            self.user(self._system_prompt)  # システムプロンプトを使用
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # 例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)  # フォーマッターを交互に適用
        for fmt, turn in zip(formatters, turns):
            fmt(turn)  # 各ターンをフォーマット
        return self
```

</div>
</details>

In [ ]:
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt  # システムプロンプトを保存
        self._few_shot_examples = few_shot_examples  # 例を保存
        # ユーザーとモデルのターンをフォーマットするためのテンプレート
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()  # 初期状態にリセット

    def __repr__(self):
        return self._state  # 現在の状態を返す

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)  # ユーザーのプロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)  # モデルのプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"  # ユーザーのターンを開始
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"  # モデルのターンを開始
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"  # ターンを終了
        return self

    def reset(self):
        self._state = ""  # 状態をクリア
        if self._system_prompt is not None:
            self.user(self._system_prompt)  # システムプロンプトを使用
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # 例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)  # フォーマッターを交互に適用
        for fmt, turn in zip(formatters, turns):
            fmt(turn)  # 各ターンをフォーマット
        return self

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def load_model(VARIANT, device):
    WEIGHTS_PATH = f'/kaggle/input/gemma/pytorch/{VARIANT}/2' 

    @contextlib.contextmanager
    def _set_default_tensor_type(dtype: torch.dtype):
        """Sets the default torch dtype to the given dtype."""
        torch.set_default_dtype(dtype)
        yield
        torch.set_default_dtype(torch.float)

    # Model Config.
    model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
    model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
    model_config.quant = "quant" in VARIANT

    # Model.
    with _set_default_tensor_type(model_config.get_dtype()):
        model = GemmaForCausalLM(model_config)
        ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{VARIANT}.ckpt')
        model.load_weights(ckpt_path)
        model = model.to(device).eval()
        
    return model
```

</div>
<div class="column-right">

# 日本語訳

```python
def load_model(VARIANT, device):
    WEIGHTS_PATH = f'/kaggle/input/gemma/pytorch/{VARIANT}/2' 

    @contextlib.contextmanager
    def _set_default_tensor_type(dtype: torch.dtype):
        """与えられたdtypeを使用してデフォルトのtorchデータ型を設定します。"""
        torch.set_default_dtype(dtype)  # デフォルトのdtypeを設定
        yield
        torch.set_default_dtype(torch.float)  # デフォルトdtypeをfloatに戻す

    # モデルの設定
    model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()  # VARIANTに応じた設定取得
    model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # トークナイザーのパスを設定
    model_config.quant = "quant" in VARIANT  # 量子化設定

    # モデルをロード
    with _set_default_tensor_type(model_config.get_dtype()):  # デフォルトのテンソル型を設定
        model = GemmaForCausalLM(model_config)  # モデルを初期化
        ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{VARIANT}.ckpt')  # チェックポイントパス
        model.load_weights(ckpt_path)  # 重みをロード
        model = model.to(device).eval()  # デバイスに移動し評価モードに設定
        
    return model  # モデルを返す
```

</div>
</details>

In [ ]:
def load_model(VARIANT, device):
    WEIGHTS_PATH = f'/kaggle/input/gemma/pytorch/{VARIANT}/2' 

    @contextlib.contextmanager
    def _set_default_tensor_type(dtype: torch.dtype):
        """与えられたdtypeを使用してデフォルトのtorchデータ型を設定します。"""
        torch.set_default_dtype(dtype)  # デフォルトのdtypeを設定
        yield
        torch.set_default_dtype(torch.float)  # デフォルトdtypeをfloatに戻す

    # モデルの設定
    model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()  # VARIANTに応じた設定取得
    model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # トークナイザーのパスを設定
    model_config.quant = "quant" in VARIANT  # 量子化設定

    # モデルをロード
    with _set_default_tensor_type(model_config.get_dtype()):  # デフォルトのテンソル型を設定
        model = GemmaForCausalLM(model_config)  # モデルを初期化
        ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{VARIANT}.ckpt')  # チェックポイントパス
        model.load_weights(ckpt_path)  # 重みをロード
        model = model.to(device).eval()  # デバイスに移動し評価モードに設定
        
    return model  # モデルを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def _parse_keyword(response: str):
    match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
    if match is None:
        keyword = ''
    else:
        keyword = match.group().lower()
    return keyword
```

</div>
<div class="column-right">

# 日本語訳

```python
def _parse_keyword(response: str):
    match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # キーワードの抽出
    if match is None:
        keyword = ''  # マッチしない場合は空文字
    else:
        keyword = match.group().lower()  # マッチした場合は小文字に変換
    return keyword  # キーワードを返す
```

</div>
</details>

In [ ]:
def _parse_keyword(response: str):
    match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # キーワードの抽出
    if match is None:
        keyword = ''  # マッチしない場合は空文字
    else:
        keyword = match.group().lower()  # マッチした場合は小文字に変換
    return keyword  # キーワードを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def _parse_response(response: str, obs: dict):
    if obs['turnType'] == 'ask':
        match = re.search(".+?\?", response.replace('*', ''))
        if match is None:
            question = "Is it a person?"
        else:
            question = match.group()
        return question
    elif obs['turnType'] == 'guess':
        guess = _parse_keyword(response)
        return guess
    else:
        raise ValueError("Unknown turn type:", obs['turnType'])
```

</div>
<div class="column-right">

# 日本語訳

```python
def _parse_response(response: str, obs: dict):
    if obs['turnType'] == 'ask':
        match = re.search(".+?\?", response.replace('*', ''))  # 質問を抽出
        if match is None:
            question = "人ですか？"  # マッチしない場合のデフォルト質問
        else:
            question = match.group()  # 質問を返す
        return question
    elif obs['turnType'] == 'guess':
        guess = _parse_keyword(response)  # キーワードを抽出
        return guess
    else:
        raise ValueError("不明なターンタイプ:", obs['turnType'])  # エラーを発生させる
```

</div>
</details>

In [ ]:
def _parse_response(response: str, obs: dict):
    if obs['turnType'] == 'ask':
        match = re.search(".+?\?", response.replace('*', ''))  # 質問を抽出
        if match is None:
            question = "人ですか？"  # マッチしない場合のデフォルト質問
        else:
            question = match.group()  # 質問を返す
        return question
    elif obs['turnType'] == 'guess':
        guess = _parse_keyword(response)  # キーワードを抽出
        return guess
    else:
        raise ValueError("不明なターンタイプ:", obs['turnType'])  # エラーを発生させる

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]
```

</div>
<div class="column-right">

# 日本語訳

```python
def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]  # 長さの異なるリストを交互にマージ
```

</div>
</details>

In [ ]:
def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]  # 長さの異なるリストを交互にマージ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a thing?", "**no**",
    "Is is a country?", "**yes**",
    "Is it Europe?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]

formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)
```

</div>
<div class="column-right">

# 日本語訳

```python
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者がはいまたはいいえで質問に答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは物ですか？", "**いいえ**",
    "それは国ですか？", "**はい**",
    "それはヨーロッパですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]

formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)  # フォーマッターを初期化
```

</div>
</details>

In [ ]:
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者がはいまたはいいえで質問に答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは物ですか？", "**いいえ**",
    "それは国ですか？", "**はい**",
    "それはヨーロッパですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]

formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)  # フォーマッターを初期化

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
obs = {
    'turnType': 'ask',
    'questions': [
        'Is it a living entity?',
        'Is it man-made?',
        'Can it be held in a single hand?'
    ],
    'answers': [
        'no',
        'yes',
        'yes'
    ]
}
```

</div>
<div class="column-right">

# 日本語訳

```python
obs = {
    'turnType': 'ask',  # 現在のターンタイプ
    'questions': [
        'それは生きているものですか？',
        '人工的なものでしょうか？',
        '片手で持てるものですか？'
    ],
    'answers': [
        'いいえ',
        'はい',
        'はい'
    ]
}  # 観察情報
```

</div>
</details>

In [ ]:
obs = {
    'turnType': 'ask',  # 現在のターンタイプ
    'questions': [
        'それは生きているものですか？',
        '人工的なものでしょうか？',
        '片手で持てるものですか？'
    ],
    'answers': [
        'いいえ',
        'はい',
        'はい'
    ]
}  # 観察情報

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
formatter.reset()
formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
turns = interleave_unequal(obs['questions'], obs['answers'])
formatter.apply_turns(turns, start_agent='model')
if obs['turnType'] == 'ask':
    formatter.user("Please ask a yes-or-no question.")
elif obs['turnType'] == 'guess':
    formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
formatter.start_model_turn()
```

</div>
<div class="column-right">

# 日本語訳

```python
formatter.reset()  # フォーマッターをリセット
formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")  # ユーザーを初期化
turns = interleave_unequal(obs['questions'], obs['answers'])  # 質問と回答を交互に整列
formatter.apply_turns(turns, start_agent='model')  # ターンを適用
if obs['turnType'] == 'ask':
    formatter.user("はいまたはいいえで答えてください。")  # 質問タイプがaskの時
elif obs['turnType'] == 'guess':
    formatter.user("今キーワードを推測してください。ダブルアスタリスクで囲んでください。")  # 質問タイプがguessの時
formatter.start_model_turn()  # モデルのターンを開始
```

</div>
</details>

In [ ]:
formatter.reset()  # フォーマッターをリセット
formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")  # ユーザーを初期化
turns = interleave_unequal(obs['questions'], obs['answers'])  # 質問と回答を交互に整列
formatter.apply_turns(turns, start_agent='model')  # ターンを適用
if obs['turnType'] == 'ask':
    formatter.user("はいまたはいいえで答えてください。")  # 質問タイプがaskの時
elif obs['turnType'] == 'guess':
    formatter.user("今キーワードを推測してください。ダブルアスタリスクで囲んでください。")  # 質問タイプがguessの時
formatter.start_model_turn()  # モデルのターンを開始

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
prompt = str(formatter)
prompt
```

</div>
<div class="column-right">

# 日本語訳

```python
prompt = str(formatter)  # フォーマッターからプロンプトを生成
prompt  # プロンプトを返す
```

</div>
</details>

In [ ]:
prompt = str(formatter)  # フォーマッターからプロンプトを生成
prompt  # プロンプトを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
MACHINE_TYPE = "cuda" 
device = torch.device(MACHINE_TYPE)
max_new_tokens = 32
sampler_kwargs = {
'temperature': 0.01,
'top_p': 0.1,
'top_k': 1,
}
```

</div>
<div class="column-right">

# 日本語訳

```python
MACHINE_TYPE = "cuda"  # 使用する機械のタイプを指定
device = torch.device(MACHINE_TYPE)  # デバイスを設定
max_new_tokens = 32  # 最大生成トークン数
sampler_kwargs = {
'temperature': 0.01,  # 温度パラメータ
'top_p': 0.1,  # 確率のトップpフィルタリング
'top_k': 1,  # トップkフィルタリング
}
```

</div>
</details>

In [ ]:
MACHINE_TYPE = "cuda"  # 使用する機械のタイプを指定
device = torch.device(MACHINE_TYPE)  # デバイスを設定
max_new_tokens = 32  # 最大生成トークン数
sampler_kwargs = {
'temperature': 0.01,  # 温度パラメータ
'top_p': 0.1,  # 確率のトップpフィルタリング
'top_k': 1,  # トップkフィルタリング
}

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Gemma 2b V2

</div>
<div class="column-right">

# 日本語訳

# Gemma 2b V2


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = load_model("2b", device)

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)
response = _parse_response(response, obs)
print(response)
```

</div>
<div class="column-right">

# 日本語訳

```python
model = load_model("2b", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力
```

</div>
</details>

In [ ]:
model = load_model("2b", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Gemma 2b-it V2

</div>
<div class="column-right">

# 日本語訳

# Gemma 2b-it V2


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = load_model("2b-it", device)

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)
response = _parse_response(response, obs)
print(response)
```

</div>
<div class="column-right">

# 日本語訳

```python
model = load_model("2b-it", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力
```

</div>
</details>

In [ ]:
model = load_model("2b-it", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Gemma 7b-it-quant V2

</div>
<div class="column-right">

# 日本語訳

# Gemma 7b-it-quant V2


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = load_model("7b-it-quant", device)

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)
response = _parse_response(response, obs)
print(response)
```

</div>
<div class="column-right">

# 日本語訳

```python
model = load_model("7b-it-quant", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力
```

</div>
</details>

In [ ]:
model = load_model("7b-it-quant", device)  # モデルをロード

response = model.generate(
    prompt,
    device=device,
    output_len=max_new_tokens,
    **sampler_kwargs
)  # モデルによる応答を生成
response = _parse_response(response, obs)  # 応答を解析
print(response)  # 応答を出力

---

# コメント

> ## dedq
> 
> コードが明確で、迅速に実行されます。Kaggleのメンバーへの最高の仕事を共有してくれてありがとう！
> 
> 
---